# Neural Networks

In [2]:
import os
os.chdir("/sfs/gpfs/tardis/home/dpv8cf/toxic-comment-analysis")  # adjust if needed
os.getcwd()


'/sfs/gpfs/tardis/home/dpv8cf/toxic-comment-analysis'

In [3]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import torch
import torch.nn as nn
import sys
sys.path.append("scripts")
import data_cleaning
from torch.utils.data import TensorDataset, DataLoader


In [4]:
data = pd.read_csv("data/train.csv")

In [5]:
data = data_cleaning.clean_data(data)

In [6]:
data.shape, data.columns

((1593229, 15),
 Index(['comment_text', 'funny', 'wow', 'sad', 'likes', 'disagree', 'toxicity',
        'severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack',
        'insult', 'threat', 'toxicity_annotator_count', 'rating_rejected'],
       dtype='object'))

## Comment Only

In [10]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# Using full dataset to create a smaller stratified sample
X_comment_only = data["comment_text"].astype(str)
y = data["rating_rejected"]

sss = StratifiedShuffleSplit(
    n_splits=1,
    train_size=100_000,        # try 50_000 if still too slow
    random_state=42
)

# Get indices for the small, stratified sample
idx_small, _ = next(sss.split(X_comment_only, y))

# Create a smaller DataFrame to run NN + param search on
data_small = data.iloc[idx_small].copy()

#Smaller Sample
X_comment_only = data_small["comment_text"].astype(str)
y = data_small["rating_rejected"]

#train/val split on the SMALL sample
X_train_comment_only, X_val_comment_only, y_train, y_val = train_test_split(
    X_comment_only,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

param_grid = []

activations = ["relu", "tanh"]
hidden_sizes = [128, 256]
lrs = [0.001, 0.01]

for act in activations:
    for h in hidden_sizes:
        for lr in lrs:
            param_grid.append({
                "activation": act,
                "hidden_size": h,
                "lr": lr
            })



In [11]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report

# ======================================================
# 1. Tokenizer
# ======================================================

def simple_tokenize(text):
    return text.lower().strip().split() #split on space

#Simple tokenizer with lowercase + split

# ======================================================
# 2. Build vocabulary from TRAIN ONLY
# ======================================================
from collections import Counter
#Counting all words appearing in training comments
word_counter = Counter()

for text in X_train_comment_only:
    tokens = simple_tokenize(text) #Convert Strings into Tokens
    word_counter.update(tokens) #Count word frequencies

max_vocab_size = 20000 #Keep only top 20k words
most_common = word_counter.most_common(max_vocab_size)
#Special Token for Padding and Unknown Words
PAD = "<PAD>"
UNK = "<UNK>"
#Creating Vocabulary Lists
idx2word = [PAD, UNK] + [w for w, _ in most_common] #Index -> Words
word2idx = {w: i for i, w in enumerate(idx2word)} #Words -> Index

pad_idx = word2idx[PAD]
unk_idx = word2idx[UNK]
vocab_size = len(word2idx)

print("Vocab size:", vocab_size)

# ======================================================
# 3. Convert text → sequence
# ======================================================

def text_to_seq(text, max_len=100):
    tokens = simple_tokenize(text) #Tokenize
    seq = [word2idx.get(tok, unk_idx) for tok in tokens] #Map to Indices
    if len(seq) < max_len:
        seq = seq + [pad_idx] * (max_len - len(seq)) #Pad at End
    else:
        seq = seq[:max_len] #Cut off Extra Words
    return np.array(seq, dtype=np.int64)
#Convert Train and Validation Into Integer Sequences
max_len = 100

X_train_seq = np.stack([text_to_seq(t) for t in X_train_comment_only])
X_val_seq   = np.stack([text_to_seq(t) for t in X_val_comment_only])

# ======================================================
# 4. Dataset + DataLoader
# ======================================================

class CommentDataset(Dataset):
    def __init__(self, X_seq, y):
        self.X = torch.from_numpy(X_seq).long() #Convert to Tensor
        self.y = torch.from_numpy(y.values).long()  # pandas Series → tensor

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64

train_dataset = CommentDataset(X_train_seq, y_train)
val_dataset   = CommentDataset(X_val_seq,   y_val)
#DataLoaders for Batching
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

# ======================================================
# 5. Neural Network
# ======================================================

def get_activation(name):
    if name == "relu":
        return nn.ReLU()
    elif name == "tanh":
        return nn.Tanh()
    else:
        raise ValueError("Invalid activation")
#Neural Network Model
class CommentNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, activation, pad_idx):
        super().__init__() #Embedding Layer: Converts Word Indices -> Dense Vectors
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embed_dim,
            padding_idx=pad_idx #Pad Tokens stay zero
        )
        self.net = nn.Sequential(
            nn.Linear(embed_dim, hidden_size),
            get_activation(activation), #ReLU or TanH
            nn.Dropout(0.3), #Regularization
            nn.Linear(hidden_size, 1),
            nn.Sigmoid() #Binary Classification 
        )

    def forward(self, x):
        # x: (batch, seq_len)
        emb = self.embedding(x)        # (batch, seq_len, embed_dim)
        pooled = emb.mean(dim=1)       # (batch, embed_dim)
        return self.net(pooled)

# ======================================================
# 6. Early Stopping Training Loop
# ======================================================

def train_model(model, optimizer, train_loader, val_loader,
                max_epochs=20, patience=3, device="cpu"):

    criterion = nn.BCELoss() #Binary Cross Entropy
    model.to(device)

    best_f1 = 0 #Track Best Validation F1
    best_state = None #Store best model weights
    wait = 0 #Patience Counter

    for epoch in range(1, max_epochs+1):
        # ---- TRAIN ----
        model.train()
        train_losses = []

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device).float().unsqueeze(1)

            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        # ---- VALIDATION ----
        model.eval()
        val_preds = []
        val_targets = []

        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)

                probs = model(xb)
                preds = (probs > 0.5).long().cpu().numpy().flatten()

                val_preds.extend(preds.tolist())
                val_targets.extend(yb.cpu().numpy().tolist())

        val_f1 = f1_score(val_targets, val_preds)
        print(f"Epoch {epoch} | Train Loss: {np.mean(train_losses):.4f} | Val F1: {val_f1:.4f}")

        # ---- EARLY STOPPING ----
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = model.state_dict()
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    # load best weights
    model.load_state_dict(best_state)
    return best_f1, model

# ======================================================
# 7. Hyperparameter Search (your param_grid)
# ======================================================

results = [] #Store Results 
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_dim = 100  # you can tune this later

for params in param_grid:
    print("="*70)
    print("Testing config:", params)
    print("="*70)
#Build Model with chosen hyperparameters
    model = CommentNN(
        vocab_size=vocab_size,
        embed_dim=embed_dim,
        hidden_size=params["hidden_size"],
        activation=params["activation"],
        pad_idx=pad_idx
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
#Train and Evaluate
    best_f1, trained_model = train_model(
        model,
        optimizer,
        train_loader,
        val_loader,
        max_epochs=20,
        patience=3,
        device=device
    )

    print(f"Best Val F1 for config {params}: {best_f1:.4f}\n")
    results.append((params, best_f1, trained_model))

# ======================================================
# 8. Best Hyperparameters
# ======================================================

best_params, best_val_f1, best_model = max(results, key=lambda x: x[1])
print("\nBest Config:", best_params)
print("Best Validation F1:", best_val_f1)


Vocab size: 20002
Testing config: {'activation': 'relu', 'hidden_size': 128, 'lr': 0.001}
Epoch 1 | Train Loss: 0.2529 | Val F1: 0.0000
Epoch 2 | Train Loss: 0.2196 | Val F1: 0.0105
Epoch 3 | Train Loss: 0.1934 | Val F1: 0.0531
Epoch 4 | Train Loss: 0.1726 | Val F1: 0.1032
Epoch 5 | Train Loss: 0.1571 | Val F1: 0.1262
Epoch 6 | Train Loss: 0.1458 | Val F1: 0.1174
Epoch 7 | Train Loss: 0.1361 | Val F1: 0.1362
Epoch 8 | Train Loss: 0.1270 | Val F1: 0.1210
Epoch 9 | Train Loss: 0.1203 | Val F1: 0.1303
Epoch 10 | Train Loss: 0.1135 | Val F1: 0.1312
Early stopping.
Best Val F1 for config {'activation': 'relu', 'hidden_size': 128, 'lr': 0.001}: 0.1362

Testing config: {'activation': 'relu', 'hidden_size': 128, 'lr': 0.01}
Epoch 1 | Train Loss: 0.2386 | Val F1: 0.0076
Epoch 2 | Train Loss: 0.1949 | Val F1: 0.0939
Epoch 3 | Train Loss: 0.1553 | Val F1: 0.0754
Epoch 4 | Train Loss: 0.1144 | Val F1: 0.1050
Epoch 5 | Train Loss: 0.0853 | Val F1: 0.1237
Epoch 6 | Train Loss: 0.0717 | Val F1: 0.139

To build a neural network that can understand raw text comments, the code first creates a smaller, balanced subset of the dataset for efficient training. Each comment is tokenized, meaning the text is converted into lowercase words split by spaces, and a vocabulary is built from the training data that assigns every word an integer ID. Because neural networks require fixed-size inputs, each comment is then transformed into a sequence of exactly 100 word indices: shorter comments are padded by adding special <PAD> tokens at the end, while longer comments are trimmed by cutting off words beyond the maximum length. These fixed-length sequences are fed into an embedding layer, which learns a dense vector representation for each word so that semantically similar words end up with similar numerical vectors. The embeddings for all words in a comment are then averaged to produce a single feature vector summarizing the entire comment. This vector is passed through a small feed-forward neural network with ReLU or Tanh activation and a final sigmoid output to predict whether the comment is toxic or rejected. The model is trained with early stopping and evaluated using validation F1 scores while searching across different hyperparameter combinations to find the best-performing architecture.

### Why Embeddings Instead of TF-IDF?

In the comment-only neural network, I used word embeddings instead of TF-IDF because embeddings allow the model to learn its own numeric representation of each word during training. TF-IDF manually assigns each word a fixed numerical weight based on frequency, but it does not capture meaning, context, or relationships between words. Embeddings, on the other hand, transform each word into a dense vector that the model continuously adjusts so that words with similar meanings end up with similar representations. In simple terms, both TF-IDF and embeddings convert text into numbers so a model can use it, but embeddings are learned automatically, while TF-IDF is hand-crafted and fixed. For raw text and neural networks, embeddings are more appropriate because they allow the model to learn richer patterns beyond simple word counts, making them a more flexible and expressive encoding method.

## Numeric Columns

In [12]:
numeric_cols = [
    "funny", "wow", "sad", "likes", "disagree",
    "toxicity", "severe_toxicity", "obscene", "sexual_explicit",
    "identity_attack", "insult", "threat", "toxicity_annotator_count"
]

In [18]:
X_num = data[numeric_cols].values.astype(np.float32)
y_num = data["rating_rejected"].values  # 0 = not toxic, 1 = toxic

In [19]:
X_train_num, X_val_num, y_train_num, y_val_num = train_test_split(
    X_num,
    y_num,
    test_size=0.2,
    random_state=42,
    stratify=y_num
)

In [20]:
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num).astype(np.float32)
X_val_num_scaled   = scaler.transform(X_val_num).astype(np.float32)

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

num_model = Pipeline([
    ("scaler", StandardScaler()),      # normalize numeric features
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"        # helps with imbalance
    ))
])


In [27]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

param_grid_num = {
    "clf__C": [0.1, 1.0, 10.0]    # low → more regularization, high → less
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_num = GridSearchCV(
    estimator=num_model,
    param_grid=param_grid_num,
    cv=cv,
    scoring="f1",
    n_jobs=-1,
    verbose=1
)

grid_num.fit(X_train_num, y_train_num)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000))]),
             n_jobs=-1, param_grid={'clf__C': [0.1, 1.0, 10.0]}, scoring='f1',
             verbose=1)

In [28]:
class NumericDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()  # 0/1

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64
train_num_dataset = NumericDataset(X_train_num_scaled, y_train_num)
val_num_dataset   = NumericDataset(X_val_num_scaled,   y_val_num)

train_num_loader = DataLoader(train_num_dataset, batch_size=batch_size, shuffle=True)
val_num_loader   = DataLoader(val_num_dataset,   batch_size=batch_size, shuffle=False)

In [29]:
def get_activation(name):
    name = name.lower()
    if name == "relu":
        return nn.ReLU()
    elif name == "tanh":
        return nn.Tanh()
    else:
        raise ValueError(f"Unknown activation: {name}")

In [30]:
class NumNN(nn.Module):
    def __init__(self, input_dim, hidden_size, activation):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_size),
            get_activation(activation),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()   # outputs P(toxic=1)
        )

    def forward(self, x):
        return self.net(x)

In [31]:
def train_num_model(model, optimizer, train_loader, val_loader,
                    max_epochs=20, patience=3, device="cpu"):

    criterion = nn.BCELoss()
    model.to(device)

    best_f1 = 0.0
    best_state = None
    wait = 0

    for epoch in range(1, max_epochs + 1):
        # ---- TRAIN ----
        model.train()
        train_losses = []

        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device).float().unsqueeze(1)  # (batch, 1)

            optimizer.zero_grad()
            probs = model(xb)                        # (batch, 1), values in [0,1]
            loss = criterion(probs, yb)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        avg_train_loss = float(np.mean(train_losses))

        # ---- VALIDATION ----
        model.eval()
        val_preds = []
        val_targets = []

        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)

                probs = model(xb)                    # (batch, 1)
                preds = (probs > 0.5).long().cpu().numpy().flatten()  # 0/1

                val_preds.extend(preds.tolist())
                val_targets.extend(yb.cpu().numpy().tolist())

        val_f1 = f1_score(val_targets, val_preds)
        print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f} | Val F1: {val_f1:.4f}")

        # ---- EARLY STOPPING ----
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = model.state_dict()
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return best_f1, model


In [32]:
param_grid_numNN = []

activations = ["relu", "tanh"]
hidden_sizes = [128, 256]
lrs = [0.001, 0.01]

for act in activations:
    for h in hidden_sizes:
        for lr in lrs:
            param_grid_numNN.append({
                "activation": act,
                "hidden_size": h,
                "lr": lr
            })

device = "cuda" if torch.cuda.is_available() else "cpu"
input_dim = X_train_num_scaled.shape[1]

results_numNN = []

for params in param_grid_numNN:
    print("=" * 70)
    print("Numbers-only NN config:", params)
    print("=" * 70)

    model = NumNN(
        input_dim=input_dim,
        hidden_size=params["hidden_size"],
        activation=params["activation"]
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

    best_f1, trained_model = train_num_model(
        model,
        optimizer,
        train_num_loader,
        val_num_loader,
        max_epochs=20,
        patience=3,
        device=device
    )

    print(f"Best Val F1 for config {params}: {best_f1:.4f}\n")
    results_numNN.append((params, best_f1, trained_model))

Numbers-only NN config: {'activation': 'relu', 'hidden_size': 128, 'lr': 0.001}
Epoch 1 | Train Loss: 0.1998 | Val F1: 0.1952
Epoch 2 | Train Loss: 0.1949 | Val F1: 0.2087
Epoch 3 | Train Loss: 0.1944 | Val F1: 0.1449
Epoch 4 | Train Loss: 0.1941 | Val F1: 0.1640
Epoch 5 | Train Loss: 0.1940 | Val F1: 0.1594
Early stopping.
Best Val F1 for config {'activation': 'relu', 'hidden_size': 128, 'lr': 0.001}: 0.2087

Numbers-only NN config: {'activation': 'relu', 'hidden_size': 128, 'lr': 0.01}
Epoch 1 | Train Loss: 0.2036 | Val F1: 0.2292
Epoch 2 | Train Loss: 0.2018 | Val F1: 0.1035
Epoch 3 | Train Loss: 0.2011 | Val F1: 0.1842
Epoch 4 | Train Loss: 0.2010 | Val F1: 0.1451
Early stopping.
Best Val F1 for config {'activation': 'relu', 'hidden_size': 128, 'lr': 0.01}: 0.2292

Numbers-only NN config: {'activation': 'relu', 'hidden_size': 256, 'lr': 0.001}
Epoch 1 | Train Loss: 0.1992 | Val F1: 0.1827
Epoch 2 | Train Loss: 0.1942 | Val F1: 0.1694
Epoch 3 | Train Loss: 0.1937 | Val F1: 0.2171
Ep

In [33]:
best_params_numNN, best_val_f1_numNN, best_model_numNN = max(
    results_numNN,
    key=lambda x: x[1]
)

print("Best numbers-only NN config:", best_params_numNN)
print("Best validation F1 (numbers-only NN):", best_val_f1_numNN)

Best numbers-only NN config: {'activation': 'tanh', 'hidden_size': 256, 'lr': 0.01}
Best validation F1 (numbers-only NN): 0.28675617437427486


## Summary 

The comment-only neural network achieved a validation F1 score of 0.17, which indicates that raw text alone is not highly predictive in this dataset. Many comments are extremely short or ambiguous, and the simple embedding + mean-pooling architecture does not capture deeper context or subtle forms of toxicity. In contrast, the numbers-only neural network—which uses reaction counts and toxicity-related metadata such as toxicity, obscene, insult, and identity_attack—performed substantially better with an F1 score of 0.29. For the numbers-only model, the best-performing hyperparameters were the Tanh activation function, a 256-unit hidden layer, and a learning rate of 0.01. These numeric features contain clearer and more direct signals associated with toxic behavior, which explains their stronger predictive power. This comparison shows that numeric metadata is more informative than raw text alone and motivates the use of combined models that integrate both text and numeric signals.